# Creating a dataframe from Toronto postcodes wikipedia page
*Added postal code longitudes & latitudes*

***
**Cut & pasted install & import commands from the NYC Neighborhood Lab**

Most aren't yet needed - but thought they'd be needed eventually, so included
***

In [3]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

***
*Include BeautifulSoup*
***

In [4]:
from bs4 import BeautifulSoup

***
*When I installed the html parsers below - each complained about missing msgpack*

*So I installed msgpack*
***

In [5]:
!conda install -c anaconda msgpack-python --yes
print("msgpack installed")

Solving environment: failed

# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/home/jupyterlab/conda/lib/python3.6/site-packages/conda/exceptions.py", line 819, in __call__
        return func(*args, **kwargs)
      File "/home/jupyterlab/conda/lib/python3.6/site-packages/conda/cli/main.py", line 78, in _main
        exit_code = do_call(args, p)
      File "/home/jupyterlab/conda/lib/python3.6/site-packages/conda/cli/conda_argparse.py", line 77, in do_call
        exit_code = getattr(module, func_name)(args, parser)
      File "/home/jupyterlab/conda/lib/python3.6/site-packages/conda/cli/main_install.py", line 11, in execute
        install(args, parser, 'install')
      File "/home/jupyterlab/conda/lib/python3.6/site-packages/conda/cli/install.py", line 235, in install
        force_reinstall=context.force,
      File "/home/jupyterlab/conda/lib/python3.6/site-packages/conda/core/solve.py", line 518, in solve_for_transact

***
*lxml - recommended html parser in the Beautiful Soup youtube video*
***

In [6]:
!pip install lxml #html parser

distributed 1.21.8 requires msgpack, which is not installed.


***
*Didn't use html5lib - so commented out*
***

In [7]:
#Alternative html parser mentioned in video - didn't use, so commented out
#!pip install html5lib #alternate html parser

***
*Access wikipedia link - 'page' contains BeautifulSoup version of wiki page*
***

In [8]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(wikipedia_link).text
page = BeautifulSoup(source, 'lxml')
#Uncomment line below to see html source page
#print(page.prettify())

***
## Main body of program
tbody contains the html table body

created 3 empty lists to add the postcode, borough, neighorhood as they are determined
the for tr loops through each of the table rows
the for td loops through the 3 items (postcode, borough, neighborhood) on each row

Once the row is parsed,
If the borough is "Not assigned" do nothing
If the neighborhood is "Not assigned" but borough isn't, set neighborhood to borough
If the borough is the same as the previous borough, add neighborhood to the last neighborhood item
Otherwise, add postcode, borough, neighborhood to their respective lists

Once the for loops complete, convert to a dataframe
***


In [9]:
tbody= page.find('tbody')
#print(tbody.prettify())

postcode_list=[]
borough_list=[]
neighborhood_list=[]
postcode=''
borough='Not assigned'
neighborhood='Not assigned'
for tr in tbody.find_all('tr'):
    i=0
    prev_borough=borough
    for td in tr.find_all('td'):
        item= (td.text)
        if i==0:
            postcode= item
        elif i==1:
            borough= item
        else:
            neighborhood= item.strip('\n')
        i+=1
    #print ("i= ", i, "postcode= ", postcode, "borough= ", borough, "neighborhood= ", neighborhood)
    
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood= borough
        if borough != prev_borough:
            postcode_list.append(postcode)
            borough_list.append(borough)
            neighborhood_list.append(neighborhood)
        else:
            neighborhood_list[-1]=neighborhood_list[-1]+", "+neighborhood
    #print(postcode_list)
postcode_df= pd.DataFrame({'Postcode':postcode_list, 'Borough':borough_list, 'Neighborhood':neighborhood_list})
postcode_df.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Victoria Village"
1,M5A,Downtown Toronto,"Harbourfront, Regent Park"
2,M6A,North York,"Lawrence Heights, Lawrence Manor"
3,M7A,Queen's Park,Queen's Park
4,M9A,Etobicoke,Islington Avenue
5,M1B,Scarborough,"Rouge, Malvern"
6,M3B,North York,Don Mills North
7,M4B,East York,"Woodbine Gardens, Parkview Hill"
8,M5B,Downtown Toronto,"Ryerson, Garden District"
9,M6B,North York,Glencairn


In [10]:
postcode_df.shape

(88, 3)

# Start of new code to add Latitude & Longitude

***
*Having trouble getting geocoder to work - decided to use csv file*
***

    

In [11]:
csv_link='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_latlong= pd.read_csv(csv_link)
df_latlong.head()
    

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_latlong.columns=['Postcode', 'Latitude', 'Longitude']
newpostal_df= postcode_df.set_index('Postcode').join(df_latlong.set_index('Postcode'), how='left', on= 'Postcode')
newpostal_df.reset_index(inplace=True)
newpostal_df.head(12)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,"Parkwoods, Victoria Village",43.753259,-79.329656
1,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
2,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
3,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
4,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
5,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
6,M3B,North York,Don Mills North,43.745906,-79.352188
7,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
8,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
9,M6B,North York,Glencairn,43.709577,-79.445073


# Start of new code for clustering & segmentation

#### Use geopy library to get the latitude and longitude values of Toronto

In [13]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geographical coordinates of Toronto are 43.653963, -79.387207.


***
*Create a map of Toronto with the neighborhoods superimposed*
***

In [14]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(newpostal_df['Latitude'], newpostal_df['Longitude'], newpostal_df['Borough'], newpostal_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

***
Let's restrict the borough to Downtown Toronto
***

In [15]:
downtown_df = newpostal_df[newpostal_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_df.head()
#downtown_df.shape

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,"Central Bay Street, Christie",43.657952,-79.387383


***
Get the geo coordinates of downtown Toronto
***

In [16]:
address = 'Downtown Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of downtown Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geographical coordinates of downtown Toronto are 43.655115, -79.380219.


***
*Overlay the neighborhoods on downtown Toronto map*
***

In [17]:
# create map of downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(downtown_df['Latitude'], downtown_df['Longitude'], downtown_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

***
*Define foursquare credentials and version*
***

In [18]:
CLIENT_ID = 'T25ZXI3SJ0PZXENRJA2H0MX3QCQ4JV4XYV010I2COSDTRIDZ' # your Foursquare ID
CLIENT_SECRET = 'QSCG0KYEFNG5LNL1SVJHUGLKTCH1N30TMCZGCPJG2ZKKL2ZE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: T25ZXI3SJ0PZXENRJA2H0MX3QCQ4JV4XYV010I2COSDTRIDZ
CLIENT_SECRET:QSCG0KYEFNG5LNL1SVJHUGLKTCH1N30TMCZGCPJG2ZKKL2ZE


***
## Explore neighborhoods in downtown Toronto - similar to Manhattan lab

*I decided to recreate the Manhattan borough clustering Lab exercise for downtown Toronto postal codes*

*I had thought of trying to group "healthy" (gym, yoga studio, park, etc.) and "unhealthy" (ice cream shop, bakery, etc.) venues together to see if I could get the postal codes*
*to cluster that way. But decided to replicate the Manhattan clustering lab to see what I'd get*
***

In [20]:
radius=500
LIMIT=100

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
downtown_venues = getNearbyVenues(names=downtown_df['Postcode'],
                                   latitudes=downtown_df['Latitude'],
                                   longitudes=downtown_df['Longitude']
                                  )


M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5S
M5T
M5V
M4W
M4X
M4Y


In [54]:
print(downtown_venues.shape)
downtown_venues.head()

(1073, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,M5A,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,M5A,43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center


In [55]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M4W,4,4,4,4,4,4
M4X,46,46,46,46,46,46
M4Y,88,88,88,88,88,88
M5A,51,51,51,51,51,51
M5B,100,100,100,100,100,100
M5C,100,100,100,100,100,100
M5E,55,55,55,55,55,55
M5G,81,81,81,81,81,81
M5H,100,100,100,100,100,100


In [56]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 201 uniques categories.


***
*Analyze each neighborhood (postal code)*
***

In [64]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood '] = downtown_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
downtown_onehot.shape

(1073, 202)

In [66]:
downtown_grouped = downtown_onehot.groupby('Neighborhood ').mean().reset_index()
downtown_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4W,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.25,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4X,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,M4Y,0.00,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.011364,0.011364,0.011364,0.000000,0.00,0.011364
3,M5A,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.019608
4,M5B,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.01,0.00,0.00,0.010000,0.000000,0.010000,0.010000,0.00,0.000000
5,M5C,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.01,0.000000
6,M5E,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,M5G,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.012346,0.000000,0.000000,0.012346,0.00,0.012346
8,M5H,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.01,0.000000
9,M5J,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.02,0.010000,0.000000,0.000000,0.010000,0.00,0.000000


In [68]:
downtown_grouped.shape

(15, 202)

In [69]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood ']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood '] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4W----
               venue  freq
0               Park  0.50
1         Playground  0.25
2              Trail  0.25
3  Accessories Store  0.00
4        Music Venue  0.00


----M4X----
                venue  freq
0          Restaurant  0.09
1         Coffee Shop  0.09
2   Indian Restaurant  0.04
3         Pizza Place  0.04
4  Italian Restaurant  0.04


----M4Y----
                 venue  freq
0  Japanese Restaurant  0.07
1          Coffee Shop  0.06
2              Gay Bar  0.05
3         Burger Joint  0.05
4     Sushi Restaurant  0.05


----M5A----
         venue  freq
0  Coffee Shop  0.18
1       Bakery  0.08
2         Park  0.06
3         Café  0.06
4   Restaurant  0.04


----M5B----
                 venue  freq
0          Coffee Shop  0.09
1       Clothing Store  0.07
2                 Café  0.04
3       Cosmetics Shop  0.04
4  Japanese Restaurant  0.03


----M5C----
            venue  freq
0     Coffee Shop  0.08
1            Café  0.06
2      Restaurant  0.05
3           Hotel 

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood ']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood '] = downtown_grouped['Neighborhood ']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Park,Playground,Trail,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
1,M4X,Restaurant,Coffee Shop,Bakery,Pub,Market,Italian Restaurant,Café,Pizza Place,Park,Indian Restaurant
2,M4Y,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Gastropub,Café,Fast Food Restaurant,Men's Store
3,M5A,Coffee Shop,Bakery,Park,Café,Mexican Restaurant,Breakfast Spot,Restaurant,Pub,Theater,Brewery
4,M5B,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Bar,Middle Eastern Restaurant,Sandwich Place,Pizza Place,Diner
5,M5C,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Cocktail Bar,Cosmetics Shop,Italian Restaurant,Gastropub,Bakery
6,M5E,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Bakery,Seafood Restaurant,Steakhouse,Cheese Shop,Café,Restaurant
7,M5G,Coffee Shop,Café,Italian Restaurant,Bar,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Sandwich Place,Ice Cream Shop,Falafel Restaurant
8,M5H,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Bar,Cosmetics Shop,Gym,Restaurant,Hotel
9,M5J,Coffee Shop,Hotel,Aquarium,Pizza Place,Café,Scenic Lookout,Italian Restaurant,Brewery,Sports Bar,Park


## Cluster neighborhoods

In [77]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood ', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 0, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

In [93]:
downtown_merged = downtown_df

# add clustering labels
downtown_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood '), on='Postcode')

downtown_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Café,Mexican Restaurant,Breakfast Spot,Restaurant,Pub,Theater,Brewery
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Bar,Middle Eastern Restaurant,Sandwich Place,Pizza Place,Diner
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Cocktail Bar,Cosmetics Shop,Italian Restaurant,Gastropub,Bakery
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Bakery,Seafood Restaurant,Steakhouse,Cheese Shop,Café,Restaurant
4,M5G,Downtown Toronto,"Central Bay Street, Christie",43.657952,-79.387383,4,Coffee Shop,Café,Italian Restaurant,Bar,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Sandwich Place,Ice Cream Shop,Falafel Restaurant


In [94]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [100]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[0] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M5B,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Bar,Middle Eastern Restaurant,Sandwich Place,Pizza Place,Diner
2,M5C,0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Cocktail Bar,Cosmetics Shop,Italian Restaurant,Gastropub,Bakery


In [101]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[0] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,1,Coffee Shop,Bakery,Park,Café,Mexican Restaurant,Breakfast Spot,Restaurant,Pub,Theater,Brewery


In [102]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[0] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M4Y,2,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Gastropub,Café,Fast Food Restaurant,Men's Store


In [103]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[0] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M4W,3,Park,Playground,Trail,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
13,M4X,3,Restaurant,Coffee Shop,Bakery,Pub,Market,Italian Restaurant,Café,Pizza Place,Park,Indian Restaurant


In [104]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[0] + list(range(5, downtown_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M5E,4,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Bakery,Seafood Restaurant,Steakhouse,Cheese Shop,Café,Restaurant
4,M5G,4,Coffee Shop,Café,Italian Restaurant,Bar,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Sandwich Place,Ice Cream Shop,Falafel Restaurant
5,M5H,4,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Bar,Cosmetics Shop,Gym,Restaurant,Hotel
6,M5J,4,Coffee Shop,Hotel,Aquarium,Pizza Place,Café,Scenic Lookout,Italian Restaurant,Brewery,Sports Bar,Park
7,M5K,4,Coffee Shop,Hotel,Café,American Restaurant,Gastropub,Restaurant,Gym,Sports Bar,Italian Restaurant,Deli / Bodega
8,M5L,4,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gastropub,Deli / Bodega,Italian Restaurant,Seafood Restaurant,Steakhouse
9,M5S,4,Café,Bar,Bakery,Restaurant,Coffee Shop,Bookstore,Japanese Restaurant,Comfort Food Restaurant,Poutine Place,Jazz Club
10,M5T,4,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Bakery,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Dumpling Restaurant,Grocery Store
11,M5V,4,Airport Lounge,Airport Terminal,Airport Service,Plane,Sculpture Garden,Boutique,Harbor / Marina,Boat or Ferry,Airport Gate,Airport


***
## Conclusions
*Clusters 0, 1 and 4 all had Coffee Shop as the most common venue. But that seems to be true across downtown.*

*Cluster 4 had Hotel 3 times in the 2nd most common venue. Maybe hotels and restraurants were clustered in Cluster 4.*

*Cluster 3 has only 2 postal codes and both include Park as a common venue.*

*Cluster 0 has only 2 postal codes and both include Clothing Shop as a common venue.*

*Clusters 1 and 2 have only 1 postal code each, so it's hard to determine why they're in their own clusters.*

*I'm curious why the clustering is so uneven. Cluster 4 has the majority of postal codes.*

***